#Dataset 3 - https://huggingface.co/datasets/boolq
Question answering

In [ ]:
from datasets import load_dataset

# loading dataset
import datasets
#train_dataset = datasets.load_dataset("boolq") if the next two lines give an error run this
train_dataset = datasets.load_dataset('boolq', split='train')
val_dataset = datasets.load_dataset('boolq', split='validation')


Using custom data configuration default
Reusing dataset boolq (/root/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5)
Using custom data configuration default
Reusing dataset boolq (/root/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5)


In [ ]:
train_dataset

Dataset({
    features: ['question', 'answer', 'passage'],
    num_rows: 9427
})

In [ ]:
d = train_dataset.to_pandas()
v = val_dataset.to_pandas()

In [ ]:
d.head(3)

,question,answer,passage
0,do iran and afghanistan speak the same language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,True,Windows Movie Maker (formerly known as Windows...


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [ ]:
d

,question,answer,passage
0,do iran and afghanistan speak the same language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,False,As with other games in The Elder Scrolls serie...
...,...,...,...
9422,is a us district court a federal court,True,The United States district courts are the gene...
9423,can a tenant get a restraining order against a...,True,"If a landlord is found to be retaliating, he o..."
9424,is the golden state warriors in the playoffs,True,The Warriors went into the 2018 playoffs as th...
9425,downton abbey will there be a season 7,False,Downton Abbey is a British period drama televi...


In [ ]:
input_boolq = d[['question', 'passage']][0:1000].dropna()
output_boolq = d['answer'][0:1000].dropna().tolist()

In [ ]:
import pandas as pd
q_list = "question: " + d['question']   +"passage" + d['passage']               
n_list = d.passage.apply(str).tolist(

dict_data = {'source_text': q_list,
      'target_text': n_list}

train = pd.DataFrame(dict_data)
train.head()      

,source_text,target_text
0,question: do iran and afghanistan speak the sa...,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,question: do good samaritan laws protect those...,Good Samaritan laws offer legal protection to ...
2,question: is windows movie maker part of windo...,Windows Movie Maker (formerly known as Windows...
3,question: is confectionary sugar the same as p...,"Powdered sugar, also called confectioners' sug..."
4,question: is elder scrolls online the same as ...,As with other games in The Elder Scrolls serie...


In [ ]:
import pandas as pd
q_list = "question: " + v['question']                  
n_list = v['passage'] + " </s>"

dict_data = {'source_text': q_list,
      'target_text': n_list}

val = pd.DataFrame(dict_data)
val.head() 

,source_text,target_text
0,question: does ethanol take more energy make t...,All biomass goes through at least some of thes...
1,question: is house tax and property tax are same,Property tax or 'house tax' is a local tax on ...
2,question: is pain experienced in a missing bod...,Phantom pain sensations are described as perce...
3,question: is harry potter and the escape from ...,Harry Potter and the Escape from Gringotts is ...
4,question: is there a difference between hydrox...,Hydroxyzine preparations require a doctor's pr...


In [ ]:
print(train.shape)
print(val.shape)

(9427, 2)
(3270, 2)


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train, test_size=0.2)

In [ ]:
%%time
!pip install --upgrade simplet5
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.3.0
    Uninstalling transformers-4.3.0:
      Successfully uninstalled transformers-4.3.0


CPU times: user 2.6 s, sys: 1.24 s, total: 3.84 s
Wall time: 14.9 s


In [ ]:
model.train(train_df = train_data,
            eval_df = val, 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=3, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
model.load_model("./simplet5-epoch-1-train-loss-2.7886-val-loss-2.6733", use_gpu=True)

In [ ]:
test_data['source_text']

8681    question: did jurassic world fallen kingdom co...
2362    question: has there ever been a host team in t...
6232    question: is it legal to have a radar detector...
1318    question: is chitty chitty bang bang just a story
543     question: has anyone won the grand slam in gol...
                              ...                        
73      question: is there such a thing as unbreakable...
1877    question: can you be called doctor with an hon...
2389    question: has anyone reached the bottom of cha...
4609              question: can i get radio 4 extra on fm
5896    question: is minor in possession of alcohol a ...
Name: source_text, Length: 1886, dtype: object

In [ ]:
q_test = test_data['source_text'][2] #pick any index from the above
q_ans = test_data['target_text'][2]
print("Question: ", q_test)
print('-'*50)
print("Answer: ",q_ans)

In [ ]:
predicted_ans = model.predict(q_test)

print(predicted_ans)